In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='7'

In [ ]:
from huggingface_hub import login

login('Your Token')

In [ ]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
checkpoint = "google/long-t5-tglobal-base"#"google/long-t5-tglobal-large"

In [ ]:
import pandas as pd
from datasets import Dataset

# Load CSV files into pandas DataFrames
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
eval_df = pd.read_csv("eval.csv")

# Filter DataFrames to retain only the required columns
train_filtered = train_df[["transcripts", "human_summaries"]]
test_filtered = test_df[["transcripts", "human_summaries"]]
eval_filtered = eval_df[["transcripts", "human_summaries"]]

# Convert pandas DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_filtered)
test_dataset = Dataset.from_pandas(test_filtered)
eval_dataset = Dataset.from_pandas(eval_filtered)

# # Verify datasets
# print("\nTrain Dataset Sample:")
# print(train_dataset[0])  # Print the first example from the train dataset

# print("\nTest Dataset Sample:")
# print(test_dataset[0])   # Print the first example from the test dataset

# print("\nEval Dataset Sample:")
# print(eval_dataset[0])   # Print the first example from the eval dataset


In [ ]:
prefix = "Summarize the given text: "
def preprocess_function(examples):
    #inputs = [prefix + doc for doc in examples["transcripts"]]
    inputs = [doc.split("""### Response:\nSummary:\n```""")[0] for doc in examples["transcripts"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["human_summaries"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_train_data = train_dataset.map(preprocess_function, batched=True)
tokenized_test_data = test_dataset.map(preprocess_function, batched=True)
tokenized_eval_data =  eval_dataset.map(preprocess_function, batched=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)#, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
model.generation_config.max_length = 128

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="long_t5_BOS",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=False,
    #lr_scheduler_type="cosine",
    #load_best_model_at_end=True,  # Load the best model at the end of training
    #metric_for_best_model="eval_loss",  # Monitor eval_loss (can be changed)
    #greater_is_better=False,  # Lower eval_loss is better
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],  # Add EarlyStoppingCallback
)

# Train the model
trainer.train()


In [ ]:
# trainer.push_to_hub()

In [ ]:
model.save_pretrained("./long_sum")
tokenizer.save_pretrained("./long_sum")

In [2]:
#### Evaluation Measurement

import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer  # Use AutoModelForSeq2SeqLM
import torch
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import csv
from tqdm.notebook import tqdm

# Check if CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
eval_df =  pd.read_csv("eval.csv")

# Filter DataFrames to retain only the required columns
train_filtered = train_df[["transcripts", "human_summaries"]]
test_filtered = test_df[["video_ids", "transcripts", "human_summaries"]]
eval_filtered = test_df[["video_ids","transcripts", "human_summaries"]]

# Convert the filtered data into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_filtered)
test_dataset = Dataset.from_pandas(test_filtered)
eval_dataset = Dataset.from_pandas(eval_filtered)

# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./long_sum").to(device)  # Use AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("./long_sum")  # Use AutoTokenizer

# Function to generate summaries
def generate_summary(model, tokenizer, dataset, max_length=128):
    model.eval()
    summaries = []

    for example in tqdm(dataset):
        inputs = tokenizer(
            example["transcripts"].split("""### Response:\nSummary:\n```""")[0],  # Add task prefix
            return_tensors="pt",
            max_length=1024,  # Adjust based on your model's max input length
            truncation=True,
            padding="max_length"
        ).to(device)  # Move inputs to GPU

        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries

# Generate summaries for the evaluation dataset
predicted_summaries = generate_summary(model, tokenizer, eval_dataset)

# Ground truths
ground_truths = eval_df["human_summaries"].tolist()

# Load metrics
rouge_metric = evaluate.load("rouge")
bert_score_metric = evaluate.load("bertscore")

# BLEU calculation function
def calculate_bleu(pred, ref):
    smoothing = SmoothingFunction().method1
    return [
        sentence_bleu([ref], pred, weights=(1, 0, 0, 0), smoothing_function=smoothing),
        sentence_bleu([ref], pred, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing),
        sentence_bleu([ref], pred, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing)
    ]

# Compute metrics
rouge_scores = rouge_metric.compute(predictions=predicted_summaries, references=ground_truths)
bleu_scores = [calculate_bleu(pred, ref) for pred, ref in zip(predicted_summaries, ground_truths)]
bert_scores = bert_score_metric.compute(predictions=predicted_summaries, references=ground_truths, lang="en")

# Summarize metrics
metrics_summary = {
    "ROUGE-1": np.mean(rouge_scores["rouge1"]),
    "ROUGE-2": np.mean(rouge_scores["rouge2"]),
    "ROUGE-L": np.mean(rouge_scores["rougeL"]),
    "BLEU-1": np.mean([score[0] for score in bleu_scores]),
    "BLEU-2": np.mean([score[1] for score in bleu_scores]),
    "BLEU-3": np.mean([score[2] for score in bleu_scores]),
    "BERT Score": np.mean(bert_scores['f1'])
}

# Print overall average scores
print("\nOverall Metrics Summary:")
for metric, score in metrics_summary.items():
    print(f"{metric}: {score:.4f}")

# Save metrics summary
summary_csv = "eval/long_sum.csv"
with open(summary_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Metric", "Score"])
    for metric, score in metrics_summary.items():
        writer.writerow([metric, score])
print(f"Metrics summary saved to {summary_csv}")

